In [1]:
import matplotlib.pyplot as plt 
import matplotlib.animation as animation
import numpy as np
from PIL import Image
from matplotlib.patches import Circle

In [2]:
def cartesian(r, theta, phi):
    
    x = r*np.sin(theta)*np.cos(phi)
    y = r*np.sin(theta)*np.sin(phi)
    z = r*np.cos(theta)
    
    return x, y, z;

def sphere(x, y, z):
    
    r = np.sqrt(x**2 + y**2 + z**2)
    theta = np.arccos(z/r)
    phi = np.arctan(x/y)
    
    return r, theta, phi;

def radian(degree):
    
    return degree*2*np.pi/360.;

In [27]:
def testgrid(theta_int, phi_int, r_source, Nx, Ny): # INPUT = intersection's coordinates OUTPUT = Color associated with the coordinates 
                                   # D'abord pour un point
    
    
    RGB = np.zeros((Ny, Nx, 3))
    
    for i in range(0, Ny):
        
        for j in range(0, Nx):
            
            if 0 <= np.cos(phi_int[i][j]) < 1 and 0 <= np.sin(phi_int[i][j]) < 1 : # Green (Q1)
                RGB[i][j] = (0, 256, 0) 

            if -1 <= np.cos(phi_int[i][j]) < 0 and 0 <= np.sin(phi_int[i][j]) < 1 :  # Red (Q1)
                RGB[i][j] = (256, 0, 0) 
    
            if -1 <= np.cos(phi_int[i][j]) < 0 and -1 <= np.sin(phi_int[i][j]) < 0 : # Yellow (Q3)
                RGB[i][j] = (256, 256, 0) 
     
            if 0 <= np.cos(phi_int[i][j]) < 1 and -1 <= np.sin(phi_int[i][j]) < 0 : #Blue (Q4)
                RGB[i][j] = (0, 0, 256) 
            
            x, y, z = cartesian(r_source, theta_int[i][j], phi_int[i][j])
                
            for k in range(0,9):
                
                y_lat = np.abs(r_source*np.sin(10*k*np.pi/180)) # 1 grille tous les 10 degrés
                error_lat = r_source/100
                
                # LATITUDE
                if  y_lat - error_lat < np.abs(y) < y_lat + error_lat :
                     RGB[i][j] = (0, 0, 0)
                    
                # LONGITUDE 
            for k in range(0, 19):     
                
                alpha_long = 10*k*np.pi/180 # sin et pas cos car sin>0 de 0 à 180 et sin. < de 180 à 360
                error_long = 2*np.pi/360 # Erreur de 1/2 radian
                alpha = np.abs(np.arctan(x/z))
                
                if alpha_long - error_long < alpha < alpha_long + error_long:      
                    RGB[i][j] = (0, 0, 0)   
                                                                                                                          
    
    
    return RGB;

In [4]:
def geodesic(M, Rs, D, N, Nx, Ny, N_rot,xmax_screen, r_source, y_pix): # Compute the intersections between the geodesics and the SCREEN and 
                                                                           # and the OBJECT
        

    # INITIAL CONDITIONS   
    
   # N_rot = 12 # Maximal number of photon orbits before it leaves the black hole's surrounding.
    dtheta = N_rot*2*np.pi/N 
    u = np.ones((Nx, N)) # Voir loop pour les CI (ATTENTION je mets 1 et pas 0 pour avoir de probleme si on divise par 0 )
    u1 = np.zeros((Nx, N)) # derivative of u et Voir loop pour les CI
    phi = np.ones((Nx, N)) 
    
    # DEFINITIONS 
    
    theta = np.zeros((Nx, N)) # the observer is located at x=0,y=0,z=D -> the first theta is 0
    theta_int = np.ones(Nx)*Rs/2  # will contain the coordinates of the intersection with the OBJECT 
    phi_int = np.ones(Nx)*Rs/2 
    phi = np.ones((Nx, N))    
    
    
    # ALGORITHM
    
    for i in range(0, Nx): 
        
        x_pix = -xmax_screen + 2*xmax_screen*i/(Nx-1) 
        di = np.sqrt(x_pix**2 + y_pix**2)
        tan0 = di/D
        
        u[i][0] = 1/D
        tampon = 0
        u1[i][0] = 1/(D*tan0)
  
        if y_pix <= 0:
           phi[i] = (-np.pi/2 + np.arcsin(x_pix/di))*phi[i]
            
        #elif y_pix == 0:
             #phi[i] = 0*phi[i]   
        
        elif y_pix > 0:
             phi[i] = (np.pi/2 - np.arcsin(x_pix/di))*phi[i]
        
        #print("pixel ", i) 
            
        angle = 0
        
        for j in range(0, N-1): 
            
            if 1/u[i][j] > Rs and tampon < 2: 
                 
                # RUNGE-KUTTA 4 FOR u1 

                f = (3/2)*Rs*u[i][j]**2 - u[i][j]
                k1 = dtheta*f
                f_k2 = (3/2)*Rs*(u[i][j] + k1/2)**2 - (u[i][j] + k1/2)
                k2 = dtheta*f_k2
                f_k3 = (3/2)*Rs*(u[i][j] + k2/2)**2 - (u[i][j] + k2/2)
                k3 = dtheta*f_k3
                f_k4 = (3/2)*Rs*(u[i][j] + k3)**2 - (u[i][j] + k3)
                k4 = dtheta*f_k4
                 
                u1[i][j+1] = u1[i][j] + k1/6 + k2/3 + k3/3 + k4/6
                
                # RUNGE-KUTTA 4 FOR u
                
                f = u1[i][j]
                k1 = dtheta*f
                f_k2 = u1[i][j] + k1/2 
                k2 = dtheta*f_k2
                f_k3 = u1[i][j] + k2/2
                k3 = dtheta*f_k3
                f_k4 = u1[i][j] + k3
                k4 = dtheta*f_k4
            
                u[i][j+1] = u[i][j] + k1/6 + k2/3 + k3/3 + k4/6
                
                # COMPUTE INTERSECTIONS VIA INTERPOLATION            
                                                    
                if 1/u[i][j] > r_source:  # Pas très précis mais bas les c
                    tampon = 2 
                    theta_int[i] = theta[i][j]
                    phi_int[i] = phi[i][j]
                    
                
                # CHANGE ANGLES AFTER HALF A REVOLUTION OF THE PHOTON
                
                if angle % 2 == 0:
                    theta[i][j+1] = theta[i][j] + dtheta
                
                if angle % 2 != 0:
                    theta[i][j+1] = theta[i][j] - dtheta  
                
                if theta[i][j+1] > np.pi: # Après un demi tour, il faut reinitialiser theta car 0 < theta < pi
                    theta[i][j+1] = np.pi
                    angle = angle + 1 
                    phi[i] = phi[i] + np.pi 
                    
                if theta[i][j+1] < 0.: # Après un demi tour, il faut reinitialiser theta car 0 < theta < pi
                    theta[i][j+1] = 0.
                    angle = angle + 1 
                    phi[i] = phi[i] + np.pi     
                
                #if angle>2:
                        #print('pixel', i)
                        #print(angle)
                           
            
            elif 1/u[i][j] <= Rs: # Si photon dans le trou noir, on arrete le calcul des geod.
                u[i][j+1] = u[i][j] 
                u1[i][j+1] = u1[i][j]   
                theta[i][j+1] = theta[i][j] 
             
                 
    return theta_int, phi_int; # return the coordinates of each intersection

In [10]:
def pixel(screen, RGB, Nx, Ny):
    
    image = np.zeros((Ny, Nx, 3))
    print(image.shape)
    
    
    for i in range(0, Ny): # i copies the index (0,1,2,3,...) and y copies 
        
        for j in range(0, Nx): #screen[4] non? 
            
            if theta_int[i][j] != Rs/2:
                image[i][j] =  RGB[Ny-1 - i][j]  # Attention les images sont créés a-pd coin superieur gaiche (0,0) avec PILLOW
                image[i][j] = image[i][j]/256
                         
      
    plt.imsave('simulation.png', image)
    
    return print("The simulation is done!");       

In [28]:
# DONNEES DE LA SIMULATION

M = 10**1 # Black hole mass
Rs = 2*M # Schwarzschild radius 
N_rot = 10 # Number of steps
N = 360*5
r_source = Rs*10**2

# CAMERA'S SETTINGS (FIXED, does not change if we approach the black hole)

D = Rs*10 # Distance between the camera and the black hole 
hfov = (120/180)*np.pi # Horizontal FoV of the camera
vfov = (120/180)*np.pi # Vertical FoV of the camera
z_screen = 0. # Where the image is formed
xmax_screen = np.tan(hfov/2)*D 
ymax_screen = np.tan(vfov/2)*D # OPTIONAL: Conserve the same aspect ratio of the background image
ratio = xmax_screen/ymax_screen

Nx = 600 # Number of x-pixel
Ny = int(Nx/ratio)    # Conserve the same aspect ratio of the background image  NOMBRE PAIR DE PIXELS!!!
print(Ny)

screen = (-xmax_screen, ymax_screen, xmax_screen, -ymax_screen)

theta_int = np.ones((Ny, Nx))
phi_int = np.ones((Ny, Nx))

for i in range(0, Ny):
    y_pix =  -ymax_screen + 2*ymax_screen*i/(Ny-1)
    print("RANGEE ------------------", i)
    theta_int[i], phi_int[i] = geodesic(M, Rs, D, N, Nx, Ny, N_rot, xmax_screen, r_source, y_pix)      

600
RANGEE ------------------ 0
RANGEE ------------------ 1
RANGEE ------------------ 2
RANGEE ------------------ 3
RANGEE ------------------ 4
RANGEE ------------------ 5
RANGEE ------------------ 6
RANGEE ------------------ 7
RANGEE ------------------ 8
RANGEE ------------------ 9
RANGEE ------------------ 10
RANGEE ------------------ 11
RANGEE ------------------ 12
RANGEE ------------------ 13
RANGEE ------------------ 14
RANGEE ------------------ 15
RANGEE ------------------ 16
RANGEE ------------------ 17
RANGEE ------------------ 18
RANGEE ------------------ 19
RANGEE ------------------ 20
RANGEE ------------------ 21
RANGEE ------------------ 22
RANGEE ------------------ 23
RANGEE ------------------ 24
RANGEE ------------------ 25
RANGEE ------------------ 26
RANGEE ------------------ 27
RANGEE ------------------ 28
RANGEE ------------------ 29
RANGEE ------------------ 30
RANGEE ------------------ 31
RANGEE ------------------ 32
RANGEE ------------------ 33
RANGEE -------------

RANGEE ------------------ 277
RANGEE ------------------ 278
RANGEE ------------------ 279
RANGEE ------------------ 280
RANGEE ------------------ 281
RANGEE ------------------ 282
RANGEE ------------------ 283
RANGEE ------------------ 284
RANGEE ------------------ 285
RANGEE ------------------ 286
RANGEE ------------------ 287
RANGEE ------------------ 288
RANGEE ------------------ 289
RANGEE ------------------ 290
RANGEE ------------------ 291
RANGEE ------------------ 292
RANGEE ------------------ 293
RANGEE ------------------ 294
RANGEE ------------------ 295
RANGEE ------------------ 296
RANGEE ------------------ 297
RANGEE ------------------ 298
RANGEE ------------------ 299
RANGEE ------------------ 300
RANGEE ------------------ 301
RANGEE ------------------ 302
RANGEE ------------------ 303
RANGEE ------------------ 304
RANGEE ------------------ 305
RANGEE ------------------ 306
RANGEE ------------------ 307
RANGEE ------------------ 308
RANGEE ------------------ 309
RANGEE ---

RANGEE ------------------ 551
RANGEE ------------------ 552
RANGEE ------------------ 553
RANGEE ------------------ 554
RANGEE ------------------ 555
RANGEE ------------------ 556
RANGEE ------------------ 557
RANGEE ------------------ 558
RANGEE ------------------ 559
RANGEE ------------------ 560
RANGEE ------------------ 561
RANGEE ------------------ 562
RANGEE ------------------ 563
RANGEE ------------------ 564
RANGEE ------------------ 565
RANGEE ------------------ 566
RANGEE ------------------ 567
RANGEE ------------------ 568
RANGEE ------------------ 569
RANGEE ------------------ 570
RANGEE ------------------ 571
RANGEE ------------------ 572
RANGEE ------------------ 573
RANGEE ------------------ 574
RANGEE ------------------ 575
RANGEE ------------------ 576
RANGEE ------------------ 577
RANGEE ------------------ 578
RANGEE ------------------ 579
RANGEE ------------------ 580
RANGEE ------------------ 581
RANGEE ------------------ 582
RANGEE ------------------ 583
RANGEE ---

In [26]:
RGB = np.zeros((Ny, Nx, 3))

RGB = testgrid(theta_int, phi_int, r_source, Nx, Ny)

pixel(screen, RGB, Nx, Ny)

(200, 200, 3)
The simulation is done!
